# Module 05: Subqueries & CTEs - Advanced Query Composition

**Estimated Time:** 60 minutes

## Learning Objectives

By the end of this module, you will be able to:
- Write subqueries in WHERE, FROM, and SELECT clauses
- Understand correlated vs non-correlated subqueries
- Use EXISTS and NOT EXISTS operators
- Create Common Table Expressions (CTEs) with WITH clause
- Use multiple CTEs in a single query
- Understand when to use subqueries vs JOINs

In [ ]:
# Setup
import sqlite3
import pandas as pd
from pathlib import Path

%load_ext sql

# Connect to database
DB_PATH = Path.cwd().parent / "data" / "databases" / "ecommerce.db"
conn = sqlite3.connect(DB_PATH)
%sql sqlite:///$DB_PATH

print("✓ Connected to ecommerce.db")

## 1. Subqueries in WHERE Clause

A subquery is a query nested inside another query. Subqueries in the WHERE clause are used for filtering.

### Types:
- **Single-value subqueries**: Return one row, one column
- **Multi-value subqueries**: Return multiple rows (use with IN, ANY, ALL)

In [ ]:
# Find products more expensive than the average price
%%sql
SELECT product_name, price
FROM products
WHERE price > (SELECT AVG(price) FROM products)
ORDER BY price DESC
LIMIT 10

In [ ]:
# Find products in the same category as 'Laptop'
%%sql
SELECT product_name, category_id, price
FROM products
WHERE category_id = (
    SELECT category_id 
    FROM products 
    WHERE product_name = 'Laptop'
)
ORDER BY price DESC

In [ ]:
# Find customers who have placed orders (using IN)
%%sql
SELECT customer_id, first_name, last_name
FROM customers
WHERE customer_id IN (
    SELECT DISTINCT customer_id 
    FROM orders
)
LIMIT 15

In [ ]:
# Find products that have never been ordered
%%sql
SELECT product_name, price
FROM products
WHERE product_id NOT IN (
    SELECT DISTINCT product_id 
    FROM order_items
)
ORDER BY price DESC
LIMIT 10

In [ ]:
# Orders with above-average total amount
%%sql
SELECT order_id, customer_id, order_date, total_amount
FROM orders
WHERE total_amount > (SELECT AVG(total_amount) FROM orders)
ORDER BY total_amount DESC
LIMIT 10

## 2. Subqueries in FROM Clause (Derived Tables)

Subqueries in FROM create temporary result sets (derived tables) that you can query.

In [ ]:
# Average order value per customer
%%sql
SELECT 
    customer_id,
    ROUND(avg_order_value, 2) AS avg_order_value,
    order_count
FROM (
    SELECT 
        customer_id,
        AVG(total_amount) AS avg_order_value,
        COUNT(*) AS order_count
    FROM orders
    GROUP BY customer_id
) AS customer_stats
WHERE order_count >= 3
ORDER BY avg_order_value DESC
LIMIT 10

In [ ]:
# Category sales summary
%%sql
SELECT 
    cat.category_name,
    sales.total_revenue,
    sales.products_sold,
    ROUND(sales.total_revenue / sales.products_sold, 2) AS avg_price_per_item
FROM categories cat
INNER JOIN (
    SELECT 
        p.category_id,
        SUM(oi.quantity * oi.unit_price) AS total_revenue,
        SUM(oi.quantity) AS products_sold
    FROM products p
    INNER JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.category_id
) AS sales ON cat.category_id = sales.category_id
ORDER BY total_revenue DESC

In [ ]:
# Top customers by spending (with ranking)
%%sql
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    cs.total_spent,
    cs.order_count
FROM customers c
INNER JOIN (
    SELECT 
        customer_id,
        SUM(total_amount) AS total_spent,
        COUNT(*) AS order_count
    FROM orders
    GROUP BY customer_id
    HAVING SUM(total_amount) > 200
) AS cs ON c.customer_id = cs.customer_id
ORDER BY cs.total_spent DESC
LIMIT 15

## 3. Subqueries in SELECT Clause

Subqueries in SELECT calculate values for each row of the outer query.

In [ ]:
# Products with their category's average price
%%sql
SELECT 
    product_name,
    price,
    (
        SELECT ROUND(AVG(price), 2)
        FROM products p2
        WHERE p2.category_id = p1.category_id
    ) AS category_avg_price
FROM products p1
ORDER BY category_id, price DESC
LIMIT 20

In [ ]:
# Customers with order count in SELECT
%%sql
SELECT 
    customer_id,
    first_name,
    last_name,
    (
        SELECT COUNT(*)
        FROM orders o
        WHERE o.customer_id = c.customer_id
    ) AS order_count
FROM customers c
ORDER BY order_count DESC
LIMIT 15

## 4. EXISTS and NOT EXISTS

EXISTS checks if a subquery returns any rows. It's more efficient than IN for large datasets.

### EXISTS vs IN:
- **EXISTS**: Returns TRUE if subquery returns any rows
- **IN**: Compares value against a list of values

In [ ]:
# Customers who have placed orders (using EXISTS)
%%sql
SELECT customer_id, first_name, last_name, email
FROM customers c
WHERE EXISTS (
    SELECT 1
    FROM orders o
    WHERE o.customer_id = c.customer_id
)
LIMIT 15

In [ ]:
# Customers who have NOT placed any orders
%%sql
SELECT customer_id, first_name, last_name, email
FROM customers c
WHERE NOT EXISTS (
    SELECT 1
    FROM orders o
    WHERE o.customer_id = c.customer_id
)
LIMIT 10

In [ ]:
# Products that have been ordered at least once
%%sql
SELECT product_id, product_name, price
FROM products p
WHERE EXISTS (
    SELECT 1
    FROM order_items oi
    WHERE oi.product_id = p.product_id
)
ORDER BY price DESC
LIMIT 15

## 5. Common Table Expressions (CTEs)

CTEs make complex queries more readable by creating named temporary result sets.

### Syntax
```sql
WITH cte_name AS (
    SELECT ...
)
SELECT * FROM cte_name;
```

In [ ]:
# Simple CTE - Customer spending
%%sql
WITH customer_totals AS (
    SELECT 
        customer_id,
        COUNT(*) AS order_count,
        SUM(total_amount) AS total_spent
    FROM orders
    GROUP BY customer_id
)
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    ct.order_count,
    ROUND(ct.total_spent, 2) AS total_spent
FROM customers c
INNER JOIN customer_totals ct ON c.customer_id = ct.customer_id
ORDER BY ct.total_spent DESC
LIMIT 10

In [ ]:
# CTE with filtering - Above average spenders
%%sql
WITH customer_spending AS (
    SELECT 
        customer_id,
        SUM(total_amount) AS total_spent
    FROM orders
    GROUP BY customer_id
),
avg_spending AS (
    SELECT AVG(total_spent) AS avg_spent
    FROM customer_spending
)
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    ROUND(cs.total_spent, 2) AS total_spent,
    ROUND(a.avg_spent, 2) AS average_customer_spent
FROM customers c
INNER JOIN customer_spending cs ON c.customer_id = cs.customer_id
CROSS JOIN avg_spending a
WHERE cs.total_spent > a.avg_spent
ORDER BY cs.total_spent DESC
LIMIT 15

## 6. Multiple CTEs

You can define multiple CTEs in a single query, separated by commas.

In [ ]:
# Multiple CTEs - Product and Category Analysis
%%sql
WITH product_sales AS (
    SELECT 
        p.product_id,
        p.product_name,
        p.category_id,
        COALESCE(SUM(oi.quantity * oi.unit_price), 0) AS revenue,
        COALESCE(SUM(oi.quantity), 0) AS units_sold
    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.product_id, p.product_name, p.category_id
),
category_totals AS (
    SELECT 
        category_id,
        SUM(revenue) AS total_category_revenue
    FROM product_sales
    GROUP BY category_id
)
SELECT 
    c.category_name,
    ps.product_name,
    ROUND(ps.revenue, 2) AS product_revenue,
    ps.units_sold,
    ROUND(ps.revenue * 100.0 / ct.total_category_revenue, 2) AS pct_of_category
FROM product_sales ps
INNER JOIN categories c ON ps.category_id = c.category_id
INNER JOIN category_totals ct ON ps.category_id = ct.category_id
WHERE ps.revenue > 0
ORDER BY c.category_name, ps.revenue DESC
LIMIT 20

In [ ]:
# Customer Segmentation with Multiple CTEs
%%sql
WITH customer_orders AS (
    SELECT 
        customer_id,
        COUNT(*) AS order_count,
        SUM(total_amount) AS total_spent,
        AVG(total_amount) AS avg_order_value,
        MAX(order_date) AS last_order_date
    FROM orders
    GROUP BY customer_id
),
customer_segments AS (
    SELECT 
        customer_id,
        order_count,
        total_spent,
        avg_order_value,
        last_order_date,
        CASE 
            WHEN total_spent >= 1000 THEN 'VIP'
            WHEN total_spent >= 500 THEN 'Gold'
            WHEN total_spent >= 200 THEN 'Silver'
            ELSE 'Bronze'
        END AS segment
    FROM customer_orders
)
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    c.email,
    cs.segment,
    cs.order_count,
    ROUND(cs.total_spent, 2) AS total_spent,
    ROUND(cs.avg_order_value, 2) AS avg_order_value,
    cs.last_order_date
FROM customers c
INNER JOIN customer_segments cs ON c.customer_id = cs.customer_id
ORDER BY cs.total_spent DESC
LIMIT 20

## 7. Real-World Examples

Complex business queries using subqueries and CTEs.

In [ ]:
# Example 1: Products performing above their category average
%%sql
WITH product_performance AS (
    SELECT 
        p.product_id,
        p.product_name,
        p.category_id,
        COALESCE(SUM(oi.quantity * oi.unit_price), 0) AS revenue
    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.product_id, p.product_name, p.category_id
),
category_avg AS (
    SELECT 
        category_id,
        AVG(revenue) AS avg_revenue
    FROM product_performance
    GROUP BY category_id
)
SELECT 
    c.category_name,
    pp.product_name,
    ROUND(pp.revenue, 2) AS revenue,
    ROUND(ca.avg_revenue, 2) AS category_avg_revenue,
    ROUND((pp.revenue - ca.avg_revenue) / ca.avg_revenue * 100, 2) AS pct_above_avg
FROM product_performance pp
INNER JOIN categories c ON pp.category_id = c.category_id
INNER JOIN category_avg ca ON pp.category_id = ca.category_id
WHERE pp.revenue > ca.avg_revenue
ORDER BY pct_above_avg DESC
LIMIT 15

In [ ]:
# Example 2: Customer Lifetime Value Analysis
%%sql
WITH customer_metrics AS (
    SELECT 
        customer_id,
        COUNT(DISTINCT order_id) AS total_orders,
        SUM(total_amount) AS lifetime_value,
        AVG(total_amount) AS avg_order_value,
        MIN(order_date) AS first_order,
        MAX(order_date) AS last_order,
        julianday(MAX(order_date)) - julianday(MIN(order_date)) AS customer_lifespan_days
    FROM orders
    GROUP BY customer_id
)
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    cm.total_orders,
    ROUND(cm.lifetime_value, 2) AS lifetime_value,
    ROUND(cm.avg_order_value, 2) AS avg_order_value,
    cm.first_order,
    cm.last_order,
    ROUND(cm.customer_lifespan_days) AS days_as_customer,
    ROUND(cm.lifetime_value / NULLIF(cm.customer_lifespan_days, 0), 2) AS value_per_day
FROM customers c
INNER JOIN customer_metrics cm ON c.customer_id = cm.customer_id
WHERE cm.customer_lifespan_days > 0
ORDER BY cm.lifetime_value DESC
LIMIT 15

## 8. Exercises

Practice what you've learned with these exercises.

### Exercise 1: Above Average Products
Find all products with a price higher than the average price of all products. Show product name, price, and the difference from average.

In [ ]:
# Your code here
%%sql

### Exercise 2: Customers Without Recent Orders
Find customers who haven't placed an order in 2024. Use NOT EXISTS or a subquery.

In [ ]:
# Your code here
%%sql

### Exercise 3: Top Spenders by Category
Using a CTE, find the customer who has spent the most in each category. Show category name, customer name, and amount spent.

In [ ]:
# Your code here
%%sql

### Exercise 4: Product Revenue Comparison
Create a query using a subquery in SELECT that shows each product with its revenue and the total revenue of all products. Calculate the percentage each product contributes.

In [ ]:
# Your code here
%%sql

### Exercise 5: Multi-CTE Customer Analysis
Using multiple CTEs, create a report showing:
- Customers who have placed more than 3 orders
- Their total spending
- Their average order value
- How they compare to the overall average order value

In [ ]:
# Your code here
%%sql

## Summary

In this module, you learned:
- ✓ Writing subqueries in WHERE, FROM, and SELECT clauses
- ✓ Understanding correlated vs non-correlated subqueries
- ✓ Using EXISTS and NOT EXISTS for efficient filtering
- ✓ Creating CTEs with WITH clause for readable queries
- ✓ Using multiple CTEs in complex analyses
- ✓ When to use subqueries vs JOINs

**Key Takeaways:**
- Subqueries can appear in WHERE, FROM, and SELECT clauses
- EXISTS is more efficient than IN for large datasets
- CTEs make complex queries more readable and maintainable
- Multiple CTEs can break down complex logic into steps
- Use subqueries when you need to compare against aggregates
- CTEs are evaluated once and reusable in the main query

**Next:** Module 06 - Data Modification

In [ ]:
# Cleanup
conn.close()
print("✓ Database connection closed")